Ссылка на репозиторий проекта:
https://github.com/andrew-tvorogov/practicum_masterskaya_2

# Маркетинг. Предсказание вероятности покупки пользователем в интернет-магазине в течение 90 дней.

<b>Описание:</b>
- Интернет-магазин собирает историю покупателей, проводит рассылки предложений и планирует будущие продажи. Для оптимизации процессов надо выделить пользователей, которые готовы совершить покупку в ближайшее время.

<b>Заказчик исследования:</b>
- yandex practicum

<b>Источник данных:</b>
- yandex practicum

<b>Содержание данных:</b>
- история покупок;
- история рекламных рассылок.

<b>Цель:</b>
- Предсказать вероятность покупки в течение 90 дней

<b>Задачи:</b>
- Изучить данные
- Разработать полезные признаки
- Создать модель для классификации пользователей
- Улучшить модель и максимизировать метрику roc_auc
- Выполнить тестирование

<b>Данные доступны в трёх файлах:</b>

- ``apparel-purchases.csv`` - история покупок
    - ``client_id`` - идентификатор пользователя
    - ``quantity`` - количество товаров в заказе
    - ``price`` - цена товара
    - ``category_ids`` - вложенные категории, к которым отнсится товар
    - ``date`` - дата покупки
    - ``message_id`` - идентификатор сообщения из рассылки

- ``apparel-messages.csv`` - история рекламных рассылок
    - ``bulk_campaign_id`` - идентификатор рекламной кампании
    - ``client_id`` - идентификатор пользователя
    - ``message_id`` - идентификатор сообщений
    - ``event`` - тип действия
    - ``channel`` - канал рассылки
    - ``date`` - дата рассылки
    - ``created_at`` - точное время создания сообщения

- ``apparel-target_binary`` - целевой бинарный признак, совершит или не совершит клиент покупку в течение следующих 90 дней
    - ``client_id`` - идентификатор пользователя
    - ``target`` - целевой признак

In [2]:
# установка пакетов

# correlation analyzer library
!pip install phik --quiet 

In [3]:
# Загрузка библиотек
import pandas as pd
import matplotlib.pyplot as plt # для диаграмм
import math # для диаграмм
import numpy as np # для равномерного распределения значений на диаграмме
import seaborn as sns
import phik

from sklearn.preprocessing import (
    LabelEncoder,
    StandardScaler,
    MinMaxScaler,
    RobustScaler, 
    OneHotEncoder, 
    OrdinalEncoder)

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, f1_score

from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SelectKBest, f_classif

from warnings import simplefilter

In [4]:
# Функции

def dataset_info(df):
    '''
    Покажет сводную информацию по датасету
    '''
    cat_columns = df.select_dtypes('object').columns # категориальные
    num_columns = df.select_dtypes('float').columns # количественные
    datetime_columns = df.select_dtypes('datetime64[ns]').columns # датавремя
    print(f'''\n''')
    df.info()
    print(f'''\n''')
    display(df.describe())
    if (len(list(datetime_columns)) > 0):
        display(startups_data[datetime_columns].agg(['min','max']))
        print(f'''\n''')
    print(f'''\n''')
    display(df.head(3))
    print(f'''\n\nЯвных дублей: {df.duplicated().sum()}\n\n''')
    print(f'''Пропущенных значений:\n{df.isna().sum()}\n\n''')

def dataset_tm_info(df):
    '''
    Покажет информацию по datetime колонкам датасета
    '''
    datetime_columns = df.select_dtypes('datetime64[ns]').columns # датавремя
    if (len(list(datetime_columns)) > 0):
        display(startups_data[datetime_columns].agg(['min','max']))
        #print(f'''\n''')
    else:
        print(f'''Нет колонок типа datetime\n''')

# функция для отображения круговой диаграммы распределения количества элементов по категории
def show_category_pie(df, field, title):
    '''
    Отобразит круговую диаграмму распределения количества элементов по категории
    '''
    df[field]\
    .value_counts().plot(kind='pie', title = title + ', %', autopct='%1.0f%%', figsize=(5,5))
    plt.ylabel('')
    plt.show()

# функция для отображения количественных признаков в виде графиков - гистограммы и ящика с усами
def show_numeric_bplt_hist(df, variable, first_label, second_label, bins):
    '''
    Отобразит количественные признаки в виде графиков - гистограммы и ящика с усами
    '''
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    ax1.hist(df[variable], bins=bins)
    ax1.set_title('Гистограмма')
    ax1.set_xlabel(first_label)
    ax1.set_ylabel(second_label)    
    ax1.grid(True)    
    
    ax2.boxplot(df[variable], vert=False)
    ax2.set_title('Ящик с усами')
    ax2.set_xlabel(first_label)
    ax2.set_ylabel(second_label)
    ax2.grid(True)
    ax2.set_yticks([])

    plt.show()
    
def show_two_barh(df_left, df_right, field, y_label, left_title, right_title):
    '''
    Отобразит две столбчатые диаграммы(горизонтальные) распределения количества элементов по категории для двух датасетов
    '''
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 4))

    df_left[field].value_counts(normalize=True).sort_values().plot.barh(alpha=0.5, color=colors[0], label='', ax=ax1)
    ax1.set_title(left_title)
    ax1.set_ylabel(y_label)    
    df_right[field].value_counts(normalize=True).sort_values().plot.barh(alpha=0.5, color=colors[1], label='', ax=ax2)    
    ax2.set_title(right_title)
    plt.show()
        
def show_two_countplots(df_left, df_right, field, y_label, x_label, left_title, right_title, y_lim=(0, 100), rotation=0, sort_field=None, prefix=''):
    '''
    Отобразит две countplot диаграммы распределения количества элементов по категории для двух датасетов
    '''
    #if (sort_field):
    #    df_left['day_index'] = df_left[sort_field] # индекс для сортировки
    #    df_left_copy = df_left.sort_values(by=f'{prefix}_{sort_field}')[field].copy()
    #    display(df_left_copy)     
    
    fig, axes = plt.subplots(1, 2)
    fig.set_size_inches(16, 4)
    
    left = sns.countplot(df_left[field], color=colors[0], ax=axes[0], alpha=0.5)
    left.set(xlabel=x_label, ylabel=y_label, title=left_title, ylim=y_lim)
    left.set_xticklabels(left.get_xticklabels(), rotation=rotation)
    
    right = sns.countplot(df_right[field], color=colors[1], ax=axes[1], alpha=0.5)
    right.set(xlabel=x_label, ylabel=y_label, title=right_title, ylim=y_lim)
    right.set_xticklabels(right.get_xticklabels(), rotation=rotation)
    
    plt.show()    

def count_lifetime(el):
    '''
    Посчитает количество дней существования стартапа
    '''
    if el['status'] == 'operating':
        return (pd.to_datetime('2018-01-01') - el["founded_at"]).days # самая "правая" дата по условию 2018-01-01
    else:
        return (el['closed_at'] - el["founded_at"]).days

def funding_per_round(value):
    '''
    Подсчитает финансирование за 1 раунд
    '''
    if value['funding_total_usd'] > 0:
        return round(value['funding_total_usd'] / value['funding_rounds'])
    else:
        return 0

def fill_nan_funding(value, funding_avg):
    '''
    Заполнит пропуски в 'funding_total_usd',
    используя величину среднего финансирования за раунд и количество раундов
    '''
    if value['funding_total_usd'] > 0:        
        return value['funding_total_usd']
    else:
        return value['funding_rounds'] * funding_avg

def fill_funding_total_usd(df):
    '''
    Заполнит пропуски в 'funding_total_usd' для датафрейма df
    '''
    df['funding_per_round'] = df.apply(lambda x: funding_per_round(x), axis=1)
    funding_median = df.query('funding_per_round > 0')['funding_per_round'].median()    
    df['funding_total_usd'] = df.apply(lambda x: fill_nan_funding(x, funding_median), axis=1)
    df = df.drop(columns=['funding_per_round'])
    return df    

In [5]:
# инициализация глобальных переменных
RANDOM_STATE = 42 # Deep Thought number
TEST_SIZE = 0.25 # размер выборки

In [6]:
# настройки
pd.options.display.max_columns = None # необходимо чтобы head отобразил все колонки
plt.rcParams['figure.figsize'] = [15, 3] # размеры графика по умолчанию
sns.set(rc={'figure.figsize':(15, 10)}) # размеры графика seaborn по умолчанию
pd.set_option('display.float_format', '{:.2f}'.format) # отобразит в колонках округленные данные до 6-ти знаков

simplefilter(action='ignore', category=FutureWarning) # уберёт предупреждение "Set `keepdims`

# цвета для диаграмм
colors = ['gray','#3672b4','#ac0936']

# Загрузка и изучение данных

## Загрузка

In [19]:
# загрузка данных в датасеты
try:
    apparel_purchases = pd.read_csv('./datasets/apparel-purchases.csv') # история покупок
    #apparel_messages = pd.read_csv('./datasets/apparel-messages.csv') # история рекламных рассылок
    apparel_messages = pd.read_csv('./datasets/apparel-messages-short.csv') # история рекламных рассылок
    apparel_target = pd.read_csv('./datasets/apparel-target_binary.csv')  # целевой признак
except:
    print("Ошибка. Данные не загружены")

### Сводная информация датасет - ``apparel_purchases``

In [20]:
dataset_info(apparel_purchases)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202208 entries, 0 to 202207
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   client_id     202208 non-null  int64  
 1   quantity      202208 non-null  int64  
 2   price         202208 non-null  float64
 3   category_ids  202208 non-null  object 
 4   date          202208 non-null  object 
 5   message_id    202208 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 9.3+ MB




,client_id,quantity,price
count,202208.00,202208.00,202208.00
mean,1515915625596210944.00,1.01,1193.30
std,145945755.93,0.18,1342.25
min,1515915625468060928.00,1.00,1.00
25%,1515915625490081280.00,1.00,352.00
50%,1515915625557177088.00,1.00,987.00
75%,1515915625624391936.00,1.00,1699.00
max,1515915626010443520.00,30.00,85499.00


,client_id,quantity,price,category_ids,date,message_id
0,1515915625468169594,1,1999.00,"['4', '28', '57', '431']",2022-05-16,1515915625468169594-4301-627b661e9736d
1,1515915625468169594,1,2499.00,"['4', '28', '57', '431']",2022-05-16,1515915625468169594-4301-627b661e9736d
2,1515915625471138230,1,6499.00,"['4', '28', '57', '431']",2022-05-16,1515915625471138230-4437-6282242f27843




Явных дублей: 73020


Пропущенных значений:
client_id       0
quantity        0
price           0
category_ids    0
date            0
message_id      0
dtype: int64




### Сводная информация датасет - ``apparel_messages``

In [21]:
dataset_info(apparel_messages)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1279999 entries, 0 to 1279998
Data columns (total 7 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   bulk_campaign_id  1279999 non-null  int64 
 1   client_id         1279999 non-null  int64 
 2   message_id        1279999 non-null  object
 3   event             1279999 non-null  object
 4   channel           1279999 non-null  object
 5   date              1279999 non-null  object
 6   created_at        1279999 non-null  object
dtypes: int64(2), object(5)
memory usage: 68.4+ MB




,bulk_campaign_id,client_id
count,1279999.00,1279999.00
mean,5265.86,1515915625532238848.00
std,476.12,75456290.62
min,548.00,1515915625468060928.00
25%,4918.00,1515915625488371456.00
50%,5299.00,1515915625491439360.00
75%,5679.00,1515915625559909120.00
max,6135.00,1515915625832435200.00


,bulk_campaign_id,client_id,message_id,event,channel,date,created_at
0,4439,1515915625626736623,1515915625626736623-4439-6283415ac07ea,open,email,2022-05-19,2022-05-19 00:14:20
1,4439,1515915625490086521,1515915625490086521-4439-62834150016dd,open,email,2022-05-19,2022-05-19 00:39:34
2,4439,1515915625553578558,1515915625553578558-4439-6283415b36b4f,open,email,2022-05-19,2022-05-19 00:51:49




Явных дублей: 29


Пропущенных значений:
bulk_campaign_id    0
client_id           0
message_id          0
event               0
channel             0
date                0
created_at          0
dtype: int64




### Сводная информация датасет - ``apparel_target``

In [22]:
dataset_info(apparel_target)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49849 entries, 0 to 49848
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   client_id  49849 non-null  int64
 1   target     49849 non-null  int64
dtypes: int64(2)
memory usage: 779.0 KB




,client_id,target
count,49849.00,49849.00
mean,1515915625598796032.00,0.02
std,148794654.89,0.14
min,1515915625468060928.00,0.00
25%,1515915625489972736.00,0.00
50%,1515915625556614912.00,0.00
75%,1515915625633408768.00,0.00
max,1515915626010443520.00,1.00


,client_id,target
0,1515915625468060902,0
1,1515915625468061003,1
2,1515915625468061099,0




Явных дублей: 0


Пропущенных значений:
client_id    0
target       0
dtype: int64


